# ওপেনবিএনবি এমসিপি সার্ভার ইন্টিগ্রেশনের সাথে সেমান্টিক কার্নেল

এই নোটবুকটি দেখায় কীভাবে সেমান্টিক কার্নেল ব্যবহার করে প্রকৃত ওপেনবিএনবি এমসিপি সার্ভারের সাথে এমসিপি স্টুডিও প্লাগইন ব্যবহার করে বাস্তব এয়ারবিএনবি থাকার জায়গা অনুসন্ধান করা যায়। এলএলএম অ্যাক্সেসের জন্য, এটি আজুর এআই ফাউন্ড্রি ব্যবহার করে। আপনার পরিবেশ ভেরিয়েবল সেটআপ করতে, আপনি [সেটআপ পাঠ](/00-course-setup/README.md) অনুসরণ করতে পারেন।


## প্রয়োজনীয় প্যাকেজগুলি আমদানি করুন


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP প্লাগইন সংযোগ তৈরি করা

আমরা MCPStdioPlugin ব্যবহার করে [OpenBnB MCP সার্ভার](https://github.com/openbnb-org/mcp-server-airbnb)-এর সাথে সংযোগ করব। এই সার্ভারটি @openbnb/mcp-server-airbnb প্যাকেজের মাধ্যমে Airbnb অনুসন্ধানের কার্যকারিতা প্রদান করে।


## ক্লায়েন্ট তৈরি করা

এই উদাহরণে, আমরা আমাদের LLM অ্যাক্সেসের জন্য Azure AI Foundry ব্যবহার করব। নিশ্চিত করুন যে আপনার পরিবেশ ভেরিয়েবলগুলি সঠিকভাবে সেটআপ করা আছে।


## পরিবেশ কনফিগারেশন

Azure OpenAI সেটিংস কনফিগার করুন। নিশ্চিত করুন যে আপনার নিম্নলিখিত পরিবেশ ভেরিয়েবলগুলি সেট করা আছে:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ইন্টিগ্রেশন বোঝা

এই নোটবুকটি **প্রকৃত OpenBnB MCP সার্ভারের** সাথে সংযুক্ত হয়, যা আসল Airbnb অনুসন্ধান কার্যকারিতা প্রদান করে।

### এটি কীভাবে কাজ করে:

1. **MCPStdioPlugin**: MCP সার্ভারের সাথে স্ট্যান্ডার্ড ইনপুট/আউটপুট যোগাযোগ ব্যবহার করে
2. **প্রকৃত NPM প্যাকেজ**: `@openbnb/mcp-server-airbnb` প্যাকেজটি npx এর মাধ্যমে ডাউনলোড এবং চালায়
3. **লাইভ ডেটা**: Airbnb এর API থেকে প্রকৃত সম্পত্তির তথ্য প্রদান করে
4. **ফাংশন আবিষ্কার**: এজেন্টটি MCP সার্ভার থেকে উপলব্ধ ফাংশনগুলি স্বয়ংক্রিয়ভাবে আবিষ্কার করে

### উপলব্ধ ফাংশনসমূহ:

OpenBnB MCP সার্ভার সাধারণত নিম্নলিখিত ফাংশনগুলি প্রদান করে:
- **search_listings** - অবস্থান এবং মানদণ্ড অনুযায়ী Airbnb সম্পত্তি অনুসন্ধান
- **get_listing_details** - নির্দিষ্ট সম্পত্তির বিস্তারিত তথ্য সংগ্রহ
- **check_availability** - নির্দিষ্ট তারিখের জন্য প্রাপ্যতা পরীক্ষা
- **get_reviews** - সম্পত্তির জন্য রিভিউ সংগ্রহ
- **get_host_info** - সম্পত্তির হোস্ট সম্পর্কে তথ্য সংগ্রহ

### পূর্বশর্ত:

- আপনার সিস্টেমে **Node.js** ইনস্টল করা থাকতে হবে
- MCP সার্ভার প্যাকেজ ডাউনলোড করার জন্য **ইন্টারনেট সংযোগ** প্রয়োজন
- **NPX** উপলব্ধ থাকতে হবে (Node.js এর সাথে আসে)

### সংযোগ পরীক্ষা:

আপনি MCP সার্ভারটি ম্যানুয়ালি পরীক্ষা করতে পারেন নিম্নলিখিত কমান্ড চালিয়ে:
```bash
npx -y @openbnb/mcp-server-airbnb
```

এটি OpenBnB MCP সার্ভারটি ডাউনলোড এবং চালু করবে, যা Semantic Kernel প্রকৃত Airbnb ডেটার জন্য সংযুক্ত হয়।


## ওপেনবিএনবি এমসিপি সার্ভারের সাথে এজেন্ট চালানো

এখন আমরা সেই এআই এজেন্টটি চালাবো, যা ওপেনবিএনবি এমসিপি সার্ভারের সাথে সংযুক্ত হয়ে স্টকহোমে ২ জন প্রাপ্তবয়স্ক এবং ১ জন শিশুর জন্য আসল এয়ারবিএনবি থাকার ব্যবস্থা খুঁজে বের করবে। অনুসন্ধানের মানদণ্ড পরিবর্তন করতে চাইলে `user_inputs` তালিকাটি পরিবর্তন করতে পারেন।


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

সারসংক্ষেপ  
অভিনন্দন! আপনি সফলভাবে একটি AI এজেন্ট তৈরি করেছেন যা Model Context Protocol (MCP) ব্যবহার করে বাস্তব জগতের আবাসন অনুসন্ধানের সাথে সংযুক্ত হয়:  

ব্যবহৃত প্রযুক্তি:  
- Semantic Kernel - Azure OpenAI ব্যবহার করে বুদ্ধিমান এজেন্ট তৈরি করার জন্য  
- Azure AI Foundry - LLM সক্ষমতা এবং চ্যাট সম্পন্ন করার জন্য  
- MCP (Model Context Protocol) - মানসম্মত টুল ইন্টিগ্রেশনের জন্য  
- OpenBnB MCP Server - বাস্তব Airbnb অনুসন্ধানের কার্যকারিতার জন্য  
- Node.js/NPX - বাইরের MCP সার্ভার চালানোর জন্য  

আপনি যা শিখেছেন:  
- MCP ইন্টিগ্রেশন: Semantic Kernel এজেন্টকে বাইরের MCP সার্ভারের সাথে সংযুক্ত করা  
- রিয়েল-টাইম ডেটা অ্যাক্সেস: লাইভ API ব্যবহার করে বাস্তব Airbnb সম্পত্তি অনুসন্ধান  
- প্রোটোকল যোগাযোগ: এজেন্ট এবং MCP সার্ভারের মধ্যে stdio যোগাযোগ ব্যবহার করা  
- ফাংশন আবিষ্কার: MCP সার্ভার থেকে উপলব্ধ ফাংশন স্বয়ংক্রিয়ভাবে আবিষ্কার করা  
- স্ট্রিমিং প্রতিক্রিয়া: রিয়েল-টাইমে ফাংশন কলগুলি ধারণ এবং লগ করা  
- HTML রেন্ডারিং: স্টাইল করা টেবিল এবং ইন্টারঅ্যাকটিভ ডিসপ্লে দিয়ে এজেন্টের প্রতিক্রিয়া ফরম্যাট করা  

পরবর্তী পদক্ষেপ:  
- অতিরিক্ত MCP সার্ভার (আবহাওয়া, ফ্লাইট, রেস্তোরাঁ) সংযুক্ত করুন  
- MCP এবং A2A প্রোটোকল একত্রিত করে একটি মাল্টি-এজেন্ট সিস্টেম তৈরি করুন  
- আপনার নিজস্ব ডেটা সোর্সের জন্য কাস্টম MCP সার্ভার তৈরি করুন  
- সেশন জুড়ে স্থায়ী কথোপকথনের স্মৃতি বাস্তবায়ন করুন  
- MCP সার্ভার অর্কেস্ট্রেশনের সাথে এজেন্টকে Azure Functions-এ ডিপ্লয় করুন  
- ব্যবহারকারীর প্রমাণীকরণ এবং বুকিং সক্ষমতা যোগ করুন  

MCP আর্কিটেকচারের প্রধান সুবিধা:  
- মানকরণ: বাইরের টুলের সাথে AI এজেন্ট সংযোগ করার জন্য একটি সার্বজনীন প্রোটোকল  
- রিয়েল-টাইম ডেটা: বিভিন্ন পরিষেবা থেকে লাইভ, আপ-টু-ডেট তথ্য অ্যাক্সেস  
- সম্প্রসারণযোগ্যতা: নতুন ডেটা সোর্স এবং টুল সহজে সংযুক্ত করার ক্ষমতা  
- আন্তঃপরিচালনযোগ্যতা: বিভিন্ন AI ফ্রেমওয়ার্ক এবং এজেন্ট প্ল্যাটফর্মে কাজ করে  
- দায়িত্বের পৃথকীকরণ: AI লজিক এবং বাইরের ডেটা অ্যাক্সেসের মধ্যে স্পষ্ট পার্থক্য  



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। নথিটির মূল ভাষায় লেখা সংস্করণটিকেই প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ ব্যবহার করার পরামর্শ দেওয়া হচ্ছে। এই অনুবাদ ব্যবহারের ফলে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
